All code pulled in reference to https://learn.deeplearning.ai/courses/building-multimodal-search-and-rag
course. For my own learning purposes I will deviate and make changes as needed.

In [4]:
# Import neural network training libraries
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms

# Import basic computation libraries along with data visualization and plotting libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.decomposition import PCA
import umap
import umap.plot
import plotly.graph_objs as go
import plotly.io as pio
pio.renderers.default = 'iframe'

# Import our data class which will organize MNIST and provide anchor, positive and negative samples.
# from mnist_dataset import MNISTDataset

/home/zhiyong/.pyenv/versions/3.10.4/envs/multimodal_rag/lib/python3.10/site-packages/numba/np/ufunc/dufunc.py:344: NumbaWarning: Compilation requested for previously compiled argument types ((uint32,)). This has no effect and perhaps indicates a bug in the calling code (compiling a ufunc more than once for the same signature
  warnings.warn(msg, errors.NumbaWarning)
/home/zhiyong/.pyenv/versions/3.10.4/envs/multimodal_rag/lib/python3.10/site-packages/numba/np/ufunc/dufunc.py:344: NumbaWarning: Compilation requested for previously compiled argument types ((uint32,)). This has no effect and perhaps indicates a bug in the calling code (compiling a ufunc more than once for the same signature
  warnings.warn(msg, errors.NumbaWarning)
/home/zhiyong/.pyenv/versions/3.10.4/envs/multimodal_rag/lib/python3.10/site-packages/numba/np/ufunc/dufunc.py:344: NumbaWarning: Compilation requested for previously compiled argument types ((uint32,)). This has no effect and perhaps indicates a bug in the ca

In [5]:
# Code copied 
import pandas as pd
from torch.utils.data import Dataset
import numpy as np
from tqdm import tqdm

class MNISTDataset(Dataset):
    def __init__(self, data_df: pd.DataFrame, transform=None, is_test=False):
        # method will run once when class object is created.
        # method will create data at the time of object creation.
        # this will save time of training
        super(MNISTDataset, self).__init__()
        dataset = []
        labels_positive = {}
        labels_negative = {}
        if is_test == False:
            # for each label create a set of same label images.
            for i in list(data_df.label.unique()):
                labels_positive[i] = data_df[data_df.label == i].to_numpy()
            # for each label create a set of image of different label.
            for i in list(data_df.label.unique()):
                labels_negative[i] = data_df[data_df.label != i].to_numpy()

        for i, row in tqdm(data_df.iterrows(), total=len(data_df)):
            data = row.to_numpy()
            # if test then only image will be returned.
            if is_test:
                label = -1
                first = data.reshape(28, 28)
                second = -1
                dis = -1
            else:
                # label and image of the index for each row in df
                label = data[0]
                first = data[1:].reshape(28, 28)
                # probability of same label image == 0.5
                if np.random.randint(0, 2) == 0:
                    # randomly select same label image
                    second = labels_positive[label][
                        np.random.randint(0, len(labels_positive[label]))
                    ]
                else:
                    # randomly select different(negative) label 
                    second = labels_negative[label][
                        np.random.randint(0, len(labels_negative[label]))
                    ]
                # cosine is 1 for same and 0 for different label
                dis = 1.0 if second[0] == label else 0.0
                # reshape image
                second = second[1:].reshape(28, 28)

            # apply transform on both images
            if transform is not None:
                first = transform(first.astype(np.float32))
                if second is not -1:
                    second = transform(second.astype(np.float32))

            # append to dataset list. 
            # this random list is created once and used in every epoch
            dataset.append((first, second, dis, label))
        
        self.dataset = dataset
        self.transform = transform
        self.is_test = is_test

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        return self.dataset[i]

<>:55: SyntaxWarning:

"is not" with a literal. Did you mean "!="?

<>:55: SyntaxWarning:

"is not" with a literal. Did you mean "!="?

/tmp/ipykernel_175938/84635887.py:55: SyntaxWarning:

"is not" with a literal. Did you mean "!="?



What's important in the creation of the dataset is this:
`dataset.append((first, second, dis, label))`
Where 
1. `first` and `second` correspond to the embeddings of the 2 objects to compare
2. `dis` is a value, either 0 for negative pair, or 1 for positive pair. We choose 1/0 because we plan to use cos similarity, hence perfect similarity is 1, not similar is 0.
3. `label` is the actual label of what the first (presumably the anchor) is. I'm not sure how this extends when we have multimodality though. I.e. we have a video of a lion and a image of a lion, is label just 'lion'? How is label actually used?

In [ ]:
# Load data from csv
data = pd.read_csv('digit-recognizer/train.csv')
val_count = 1000
# common transformation for both val and train
default_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5),
    transforms.Lambda(torch.flatten) # added a flatten cause i felt this makes more sense in a multi modal context, since we would have language embeddings (normally 1D)
])

# Split data into val and train
dataset = MNISTDataset(data.iloc[:-val_count], default_transform)
val_dataset = MNISTDataset(data.iloc[-val_count:], default_transform)

100%|██████████| 1000/1000 [00:00<00:00, 2354.94it/s]


In [39]:
# Create torch dataloaders
trainLoader = DataLoader(
    dataset,
    batch_size=16, # feel free to modify this value
    shuffle=True,
    pin_memory=True,
    num_workers=2,
    prefetch_factor=100
)

valLoader = DataLoader(val_dataset,
    batch_size=64,
    shuffle=True,
    pin_memory=True,
    num_workers=2,
    prefetch_factor=100
)

In [ ]:
sample = next(iter(trainLoader))
# batch size x rgb channels x row x col for the 2 images.
sample[0].shape, sample[1].shape, sample[2], sample[3]

(torch.Size([16, 784]),
 torch.Size([16, 784]),
 tensor([0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        dtype=torch.float64),
 tensor([2, 5, 8, 1, 0, 1, 4, 6, 4, 3, 8, 0, 8, 1, 1, 3]))